In [1]:
import pandas as pd
import numpy as np

import os
import sys
import pathlib

# librosa is a Python library for analyzing audio and music. It can be used to extract the data from the audio files we will see it later.
import matplotlib.pyplot as plt
import librosa
import librosa.display
import seaborn as sns

from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# to play the audio files
from IPython.display import Audio
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
# from tensorflow.keras.utils import np_utils, to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [17]:
data_folder = pathlib.Path("Dataset/")
# Paths for data.
Ravdess = os.path.join(data_folder, "RAVDESS Emotional speech audio", "audio_speech_actors_01-24")
Crema = os.path.join(data_folder, "CREMA-D", "AudioWAV")
Tess = os.path.join(data_folder, "Toronto emotional speech set (TESS)", "tess toronto emotional speech set data", "TESS Toronto emotional speech set data")
Savee = os.path.join(data_folder, "Surrey Audio-Visual Expressed Emotion (SAVEE)", "ALL")

In [20]:
ravdess_directory_list = os.listdir(Ravdess)

file_emotion = []
file_path = []
for dir in ravdess_directory_list:
    # as their are 20 different actors in our previous directory we need to extract files for each actor.
    actor = os.listdir(os.path.join(Ravdess, dir))
    for file in actor:
        part = file.split('.')[0]
        part = part.split('-')
        # third part in each file represents the emotion associated to that file.
        file_emotion.append(int(part[2]))
        file_path.append(os.path.join(Ravdess, dir, file))
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Ravdess_df = pd.concat([emotion_df, path_df], axis=1)

# changing integers to actual emotions.
Ravdess_df.Emotions.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
Ravdess_df.head()

,Emotions,Path
0,neutral,Dataset\RAVDESS Emotional speech audio\audio_s...
1,neutral,Dataset\RAVDESS Emotional speech audio\audio_s...
2,neutral,Dataset\RAVDESS Emotional speech audio\audio_s...
3,neutral,Dataset\RAVDESS Emotional speech audio\audio_s...
4,calm,Dataset\RAVDESS Emotional speech audio\audio_s...


In [21]:
crema_directory_list = os.listdir(Crema)

file_emotion = []
file_path = []

for file in crema_directory_list:
    # storing file paths
    file_path.append(os.path.join(Crema, file))
    # storing file emotions
    part=file.split('_')
    if part[2] == 'SAD':
        file_emotion.append('sad')
    elif part[2] == 'ANG':
        file_emotion.append('angry')
    elif part[2] == 'DIS':
        file_emotion.append('disgust')
    elif part[2] == 'FEA':
        file_emotion.append('fear')
    elif part[2] == 'HAP':
        file_emotion.append('happy')
    elif part[2] == 'NEU':
        file_emotion.append('neutral')
    else:
        file_emotion.append('Unknown')
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Crema_df = pd.concat([emotion_df, path_df], axis=1)
Crema_df.head()

,Emotions,Path
0,angry,Dataset\CREMA-D\AudioWAV\1001_DFA_ANG_XX.wav
1,disgust,Dataset\CREMA-D\AudioWAV\1001_DFA_DIS_XX.wav
2,fear,Dataset\CREMA-D\AudioWAV\1001_DFA_FEA_XX.wav
3,happy,Dataset\CREMA-D\AudioWAV\1001_DFA_HAP_XX.wav
4,neutral,Dataset\CREMA-D\AudioWAV\1001_DFA_NEU_XX.wav


In [22]:
tess_directory_list = os.listdir(Tess)

file_emotion = []
file_path = []

for dir in tess_directory_list:
    directories = os.listdir(os.path.join(Tess, dir))
    for file in directories:
        part = file.split('.')[0]
        part = part.split('_')[2]
        if part=='ps':
            file_emotion.append('surprise')
        else:
            file_emotion.append(part)
        file_path.append(os.path.join(Tess, dir, file))
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Tess_df = pd.concat([emotion_df, path_df], axis=1)
Tess_df.head()

,Emotions,Path
0,angry,Dataset\Toronto emotional speech set (TESS)\te...
1,angry,Dataset\Toronto emotional speech set (TESS)\te...
2,angry,Dataset\Toronto emotional speech set (TESS)\te...
3,angry,Dataset\Toronto emotional speech set (TESS)\te...
4,angry,Dataset\Toronto emotional speech set (TESS)\te...


In [23]:
savee_directory_list = os.listdir(Savee)

file_emotion = []
file_path = []

for file in savee_directory_list:
    file_path.append(os.path.join(Savee, file))
    part = file.split('_')[1]
    ele = part[:-6]
    if ele=='a':
        file_emotion.append('angry')
    elif ele=='d':
        file_emotion.append('disgust')
    elif ele=='f':
        file_emotion.append('fear')
    elif ele=='h':
        file_emotion.append('happy')
    elif ele=='n':
        file_emotion.append('neutral')
    elif ele=='sa':
        file_emotion.append('sad')
    else:
        file_emotion.append('surprise')
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Savee_df = pd.concat([emotion_df, path_df], axis=1)
Savee_df.head()

,Emotions,Path
0,angry,Dataset\Surrey Audio-Visual Expressed Emotion ...
1,angry,Dataset\Surrey Audio-Visual Expressed Emotion ...
2,angry,Dataset\Surrey Audio-Visual Expressed Emotion ...
3,angry,Dataset\Surrey Audio-Visual Expressed Emotion ...
4,angry,Dataset\Surrey Audio-Visual Expressed Emotion ...


In [72]:
# creating Dataframe using all the 4 dataframes we created so far.
data_path = pd.concat([Ravdess_df, Crema_df, Tess_df, Savee_df], axis = 0)
data_path.to_csv("data_path.csv",index=False)
data_path.head()
path_ds = tf.convert_to_tensor(data_path.Path)
emotions = data_path.Emotions.unique()
path_ds, emotions

(<tf.Tensor: shape=(12162,), dtype=string, numpy=
 array([b'Dataset\\RAVDESS Emotional speech audio\\audio_speech_actors_01-24\\Actor_01\\03-01-01-01-01-01-01.wav',
        b'Dataset\\RAVDESS Emotional speech audio\\audio_speech_actors_01-24\\Actor_01\\03-01-01-01-01-02-01.wav',
        b'Dataset\\RAVDESS Emotional speech audio\\audio_speech_actors_01-24\\Actor_01\\03-01-01-01-02-01-01.wav',
        ...,
        b'Dataset\\Surrey Audio-Visual Expressed Emotion (SAVEE)\\ALL\\KL_su13.wav',
        b'Dataset\\Surrey Audio-Visual Expressed Emotion (SAVEE)\\ALL\\KL_su14.wav',
        b'Dataset\\Surrey Audio-Visual Expressed Emotion (SAVEE)\\ALL\\KL_su15.wav'],
       dtype=object)>,
 array(['neutral', 'calm', 'happy', 'sad', 'angry', 'fear', 'disgust',
        'surprise'], dtype=object))

In [29]:
path_ds = tf.random.shuffle(path_ds)

In [3]:
def noise(data):
    noise_amp = 0.035*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

def stretch(data, rate=0.8):
    return librosa.effects.time_stretch(data, rate=rate)

def shift(data):
    shift_range = int(np.random.uniform(low=-5, high = 5)*1000)
    return np.roll(data, shift_range)

def pitch(data, sampling_rate, pitch_factor=0.7):
    # return librosa.effects.pitch_shift(data, sr=sampling_rate, pitch_factor)
    return librosa.effects.pitch_shift(data, sr=sampling_rate, n_steps=pitch_factor)


In [4]:
def extract_features(data):
    # ZCR
    result = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result=np.hstack((result, zcr)) # stacking horizontally

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chroma_stft)) # stacking horizontally

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc)) # stacking horizontally

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms)) # stacking horizontally

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel)) # stacking horizontally
    
    return result

def get_features(path):
    # duration and offset are used to take care of the no audio in start and the ending of each audio files as seen above.
    data, sample_rate = librosa.load(path, duration=2.5, offset=0.6)
    
    # without augmentation
    res1 = extract_features(data)
    result = np.array(res1)
    
    # data with noise
    noise_data = noise(data)
    res2 = extract_features(noise_data)
    result = np.vstack((result, res2)) # stacking vertically
    
    # data with stretching and pitching
    new_data = stretch(data)
    data_stretch_pitch = pitch(new_data, sample_rate)
    res3 = extract_features(data_stretch_pitch)
    result = np.vstack((result, res3)) # stacking vertically
    
    return result

In [53]:

def decode_audio(path):
  # Decode WAV-encoded audio files to `float32` tensors, normalized
  # to the [-1.0, 1.0] range. Return `float32` audio and a sample rate.
  audio, sample_rate = librosa.load(path, duration=2.5, offset=0.6)
  audio = librosa.util.normalize(audio)
  # Since all the data is single channel (mono), drop the `channels`
  # axis from the array.
  return tf.squeeze(audio, axis=-1)

def get_label(file_path):
  parts = tf.strings.split(
      input=file_path,
      sep=os.path.sep)
  # Note: You'll use indexing here instead of tuple unpacking to enable this
  # to work in a TensorFlow graph.
  return parts[-2]

def get_waveform_and_label(file_path):
  sess = tf.compat.v1.Session()
  waveform = decode_audio(sess.run(file_path))
  label = get_label(file_path)
  return waveform, label

def get_spectrogram(waveform):
  # Zero-padding for an audio waveform with less than 16,000 samples.
  # input_len = 50000
  # waveform = waveform[:input_len]
  # zero_padding = tf.zeros(
  #     [50000] - tf.shape(waveform),
  #     dtype=tf.float32)
  # Cast the waveform tensors' dtype to float32.
  waveform = tf.cast(waveform, dtype=tf.float32)
  # Concatenate the waveform with `zero_padding`, which ensures all audio
  # clips are of the same length.
  # equal_length = tf.concat([waveform, zero_padding], 0)
  # Convert the waveform to a spectrogram via a STFT.
  spectrogram = tf.signal.stft(
      waveform, frame_length=255, frame_step=128)
  # Obtain the magnitude of the STFT.
  spectrogram = tf.abs(spectrogram)
  # Add a `channels` dimension, so that the spectrogram can be used
  # as image-like input data with convolution layers (which expect
  # shape (`batch_size`, `height`, `width`, `channels`).
  spectrogram = spectrogram[..., tf.newaxis]
  return spectrogram

def plot_spectrogram(spectrogram, ax):
  if len(spectrogram.shape) > 2:
    assert len(spectrogram.shape) == 3
    spectrogram = np.squeeze(spectrogram, axis=-1)
  # Convert the frequencies to log scale and transpose, so that the time is
  # represented on the x-axis (columns).
  # Add an epsilon to avoid taking a log of zero.
  log_spec = np.log(spectrogram.T + np.finfo(float).eps)
  height = log_spec.shape[0]
  width = log_spec.shape[1]
  X = np.linspace(0, np.size(spectrogram), num=width, dtype=int)
  Y = range(height)
  ax.pcolormesh(X, Y, log_spec)

In [6]:
# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [18]:
data_dir = pathlib.Path('Dataset/') / "data"
filenames = tf.io.gfile.glob(str(data_dir) + '/*/*')
filenames = tf.random.shuffle(filenames)
print(filenames)

tf.Tensor(
[b'Dataset\\data\\disgust\\1053_TSI_DIS_XX.wav'
 b'Dataset\\data\\disgust\\1003_IEO_DIS_MD.wav'
 b'Dataset\\data\\happy\\OAF_tell_happy.wav' ...
 b'Dataset\\data\\disgust\\1064_TIE_DIS_XX.wav'
 b'Dataset\\data\\happy\\YAF_fat_happy.wav'
 b'Dataset\\data\\surprise\\YAF_lot_ps.wav'], shape=(12162,), dtype=string)


In [54]:
# Create dataset for training
AUTOTUNE = tf.data.AUTOTUNE

files_ds = tf.data.Dataset.from_tensor_slices(filenames)
# for data in files_ds.take(1):
#     print(data)
waveform_ds = files_ds.map(
    map_func=get_waveform_and_label,
    num_parallel_calls=AUTOTUNE)

InvalidArgumentError: in user code:

    File "C:\Users\abdul\AppData\Local\Temp\ipykernel_12752\3080063963.py", line 20, in get_waveform_and_label  *
        waveform = decode_audio(sess.run(file_path))

    InvalidArgumentError: Graph execution error:
    
    Detected at node 'args_0' defined at (most recent call last):
        File "<frozen runpy>", line 198, in _run_module_as_main
        File "<frozen runpy>", line 88, in _run_code
        File "C:\Users\abdul\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
          app.launch_new_instance()
        File "C:\Users\abdul\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance
          app.start()
        File "C:\Users\abdul\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 725, in start
          self.io_loop.start()
        File "c:\Users\abdul\AppData\Local\Programs\Python\Python311\Lib\site-packages\t